In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from mnist import MNIST

mndata = MNIST('samples')

x_train, y_train = mndata.load_training()

x_test, y_test = mndata.load_testing()

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
y_train = pd.get_dummies(y_train)
y_train = np.array(y_train)

y_test = pd.get_dummies(y_test)
y_test = np.array(y_test)

y_test and x_test are from the original test data and x_val,y_val below are the validation datasets of size 6000

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [ ]:
class neuralnet_phases():
    def __init__(self,x_train,y_train):
        
        hiddenlayernodes = 130
        self.learning_rate = 5
        self.epoch_no = 1   # Counter for number of epochs 
        attributes = 784    # Input features
        labels = 10         # Lables for numberes 0-9
        
        
        self.w1 = np.random.randn(attributes,hiddenlayernodes)
        self.b1 = np.zeros((1,hiddenlayernodes))
    
        
        self.w2 = np.random.randn(hiddenlayernodes,labels)
        self.b2 = np.zeros((1,labels))
        
        
        self.x_train = x_train
        self.y_train = y_train
        
    def feedforward(self):
        z1 = np.dot(self.x_train,self.w1) + self.b1
        self.layer1 = 1/(1 + np.exp(-z1))          
        #sigmoid given as (1/1+ e^-z)
        
        z2 = np.dot(self.layer1,self.w2) + self.b2
        exps = np.exp(z2 - np.max(z2, axis=1, keepdims=True))
        self.layer2 = exps/np.sum(exps, axis=1, keepdims=True)
        # softmax given as (e^(z-max(z)/summation of z))
        
    def backpropogation(self):

        logLik = - np.log(self.layer2[np.arange(self.y_train.shape[0]), self.y_train.argmax(axis=1)])
        loss = np.sum(logLik)/self.y_train.shape[0]
        
        print("The Loss after epoch ",self.epoch_no," is: ",loss)
        
        self.epoch_no += 1
        
        #no derivative for z3 as explaied before.
        layer2dt = ((self.layer2 - self.y_train)/(self.y_train.shape[0]))
        
        z1dt = np.dot(layer2dt,self.w2.T)
        layer1dt = z1dt * (self.layer1 * (1 - self.layer1)) # sigmoid derivative given as (x * (1-x))
        
        
        self.w2 -= self.learning_rate * np.dot(self.layer1.T,layer2dt)
        self.b2 -= self.learning_rate * np.sum(layer2dt , axis = 0, keepdims = True)
        
        
        self.w1 -= self.learning_rate * np.dot(self.x_train.T,layer1dt)
        self.b1 -= self.learning_rate * np.sum(layer1dt, axis = 0 , keepdims = True)
        
    
    def epochoutput(self,train):
        self.x_train = train
        self.feedforward()
        return self.layer2.argmax()
    
fullyconnectedlayer = neuralnet_phases(x_train, y_train)

for i in range(1000):
    fullyconnectedlayer.feedforward()
    fullyconnectedlayer.backpropogation()    
    
print("Done!")

def accuracy(train,test):
    accuracy_counter = 0
    for x,y in zip(train, test):
        max_value_from_last_layer = fullyconnectedlayer.epochoutput(x)
        if max_value_from_last_layer == np.argmax(y):
            accuracy_counter +=1
    return accuracy_counter/len(train)*100

print("Training accuracy : ", accuracy(x_train, y_train))
# print("Test accuracy : ", accuracy(x_val, y_val))
print("Actual Test Accuracy: ",accuracy(x_test,y_test))